<a href="https://www.kaggle.com/code/mikedelong/turkish-earthquakes-on-a-map?scriptVersionId=220643749" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
df = pd.read_csv(filepath_or_buffer='/kaggle/input/turkey-earthquake-data-1914-2023/veriler.csv', parse_dates=['Olus tarihi'], index_col=[0])
df['year'] = df['Olus tarihi'].dt.year
df['xm_int'] = df['xM'].apply(round, args=(0, ))
df.head()

,Deprem Kodu,Olus tarihi,Olus zamani,Enlem,Boylam,Der(km),xM,MD,ML,Mw,Ms,Mb,Tip,Yer,year,xm_int
No,,,,,,,,,,,,,,,,
1,20230731215338,2023-07-31,21:53:38.29,37.4153,37.1703,4.0,3.5,0.0,3.5,3.4,0.0,0.0,Ke,DOGANLI-PAZARCIK (KAHRAMANMARAS) [East 1.3 km],2023,4.0
2,20230731152316,2023-07-31,15:23:16.85,38.1413,38.4340,5.2,3.5,0.0,3.5,3.4,0.0,0.0,Ke,AKSU-SINCIK (ADIYAMAN) [West 6.4 km],2023,4.0
3,20230731025049,2023-07-31,02:50:49.84,37.8352,26.8353,6.2,3.0,0.0,3.0,0.0,0.0,0.0,Ke,SISAM ADASI (EGE DENIZI),2023,3.0
4,20230730201940,2023-07-30,20:19:40.49,40.8085,31.0183,5.3,3.1,0.0,3.1,3.1,0.0,0.0,Ke,ICMELER-GOLYAKA (DUZCE) [South East 0.7 km],2023,3.0
5,20230730161352,2023-07-30,16:13:52.62,38.1290,38.1863,2.4,3.0,0.0,3.0,0.0,0.0,0.0,Ke,CAMPINAR-CELIKHAN (ADIYAMAN) [North West 4.3 km],2023,3.0


In [2]:
from plotly import express
from plotly.offline import init_notebook_mode

init_notebook_mode(connected=True)
express.histogram(data_frame=df.nunique().to_frame().reset_index().rename(columns={'index': 'column', 0: 'unique values'}), x='column', y='unique values').show(renderer='iframe_connected')

In [3]:
df['Tip'].value_counts().to_frame().T

Tip,Ke,Sm
count,49886,114


In [4]:
express.scatter_mapbox(mapbox_style='open-street-map', data_frame=df[df['xM'] > 3.1], lat='Enlem', lon='Boylam', color='xM', hover_name='Yer', zoom=5, height=900).show(renderer='iframe_connected')

From the data card we know that this dataset contains earthquakes magnitude 3 or greater, so xM must be magnitude. We have floored the xM here for performance reasons.

In [5]:
# what percentage of the data do we see in the map above?
round(100 * len(df[df['xM'] >= 3.1])/len(df), 2)

78.27

In [6]:
express.histogram(data_frame=df, x='Boylam', color='xm_int').show(renderer='iframe_connected')

In [7]:
express.histogram(data_frame=df, y='Enlem', color='xm_int').show(renderer='iframe_connected')

These two charts tell us the density in the lat and lon directions and remind us that magnitude is a log scale.

In [8]:
express.pie(data_frame=df[['xm_int']], names='xm_int' ).show(renderer='iframe_connected')

If we look at our other M columns (graphs not shown) it becomes clear that our xM data is rolled up from five sources, each represented in a separate column.

In [9]:
express.histogram(data_frame=df, x='xM', color='year').show(renderer='iframe_connected')

This distribution is roughly what we would expect for a logarithmic measure.

In [10]:
express.histogram(data_frame=df, x='Deprem Kodu', color='year').show(renderer='iframe_connected')

Our first column is clearly an integer timestamp.

In [11]:
express.scatter(data_frame=df, x='year', y='xM', color='xM', trendline='ols').show(renderer='iframe_connected')

Yes it does look like earthquakes are getting more severe. From the looks of the OLS coefficient it is getting worse by about 0.8% per year, which is nontrivial given the nature of xM.